In [1]:
# Mengimpor library yang diperlukan
import pandas as pd
from google.colab import drive
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import random
import json
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_path = '/content/drive/My Drive/UAS-GANJIL-2024/NLP_Kelompok 5/patterns_responses.csv'

In [4]:
# Membaca dataset ke dalam DataFrame menggunakan Pandas
df = pd.read_csv(dataset_path)
# Tampilkan baris pertama dari dataset
print(df.head(50))

   Pattern                                           Response
0      NaN                                    Maaf tidak tahu
1      NaN                                    saya tidak tahu
2      NaN                         kata diluar kemampuan kami
3      NaN  kami tidak tahu, mungkin anda bisa menanyakan ...
4      NaN  mungkin anda bisa menanyakan mengenai aplikasi...
5      NaN                                    Maaf tidak tahu
6      NaN                                    saya tidak tahu
7      NaN                         kata diluar kemampuan kami
8      NaN  kami tidak tahu, mungkin anda bisa menanyakan ...
9      NaN  mungkin anda bisa menanyakan mengenai aplikasi...
10     NaN                                    Maaf tidak tahu
11     NaN                                    saya tidak tahu
12     NaN                         kata diluar kemampuan kami
13     NaN  kami tidak tahu, mungkin anda bisa menanyakan ...
14     NaN  mungkin anda bisa menanyakan mengenai aplikasi...
15     N

Memeriksa Struktur Dataset
Setelah memuat dataset, penting untuk memeriksa struktur dan isi dari dataset tersebut.

In [5]:
# Membaca dataset ke dalam DataFrame
data = pd.read_csv(dataset_path)

# Menampilkan nama kolom yang ada dalam dataset
print("Nama kolom dalam dataset:")
print(data.columns)

Nama kolom dalam dataset:
Index(['Pattern', 'Response'], dtype='object')


- Menyiapkan Data untuk Model dengan memisahkan kolom patterns dan tag untuk digunakan dalam pelatihan model. Kita juga akan melakukan lemmatization pada teks untuk mengurangi variasi kata.

- menggunakan WordNetLemmatizer dari NLTK untuk melakukan lemmatization. Namun, karena kita menggunakan bahasa Indonesia, kita akan menggunakan pendekatan yang lebih sederhana untuk membersihkan teks.

- Membuat Dataset untuk Model, dua list: satu untuk patterns dan satu untuk tags.

In [6]:
import pandas as pd
from nltk.stem import WordNetLemmatizer

# Inisialisasi lemmatizer
lemmatizer = WordNetLemmatizer()

# Menyiapkan data
patterns = []
responses = []

# Mengisi list patterns dan responses
for index, row in data.iterrows():
    if isinstance(row['Pattern'], str):  # Periksa apakah row['pattern'] adalah string
        for pattern in row['Pattern'].split('|'):  # Menggunakan 'pattern' sesuai nama kolom
            patterns.append(pattern)
            responses.append(row['Response'])  # Menggunakan 'response' sesuai nama kolom
    else:
        # Tangani jika nilai pattern bukan string (misalnya NaN atau float)
        patterns.append(None)
        responses.append(row['Response'])  # Menggunakan 'response' sesuai nama kolom

# Membuat DataFrame dari patterns dan responses
training_data = pd.DataFrame({'Patterns': patterns, 'Responses': responses})

# Menampilkan baris dari training_data
print(training_data.head(100))

         Patterns                                          Responses
0            None                                    Maaf tidak tahu
1            None                                    saya tidak tahu
2            None                         kata diluar kemampuan kami
3            None  kami tidak tahu, mungkin anda bisa menanyakan ...
4            None  mungkin anda bisa menanyakan mengenai aplikasi...
..            ...                                                ...
95  Selamat malam          Apakah Anda membutuhkan bantuan hari ini?
96  Selamat malam                            halo, silahkan bertanya
97  Selamat malam  Saya hanya program komputer, tetapi siap memba...
98  Selamat malam  Silahkan Jangan ragu untuk bertanya jika ada y...
99  Selamat malam           Tentu, saya di sini untuk membantu Anda.

[100 rows x 2 columns]


memisahkan data menjadi set pelatihan dan set pengujian menggunakan kode berikut:

In [8]:
from sklearn.model_selection import train_test_split

# Memisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(training_data['Patterns'], training_data['Responses'], test_size=0.2, random_state=42)

# Menampilkan ukuran set pelatihan dan pengujian
print(f'Ukuran set pelatihan: {len(X_train)}')
print(f'Ukuran set pengujian: {len(X_test)}')

Ukuran set pelatihan: 534
Ukuran set pengujian: 134


In [9]:
import pandas as pd

# Menampilkan jumlah nilai NaN/None pada X_train
print("Jumlah nilai NaN pada X_train:", pd.Series(X_train).isnull().sum())

# Menampilkan jumlah nilai NaN/None pada y_train
print("Jumlah nilai NaN pada y_train:", pd.Series(y_train).isnull().sum())

Jumlah nilai NaN pada X_train: 20
Jumlah nilai NaN pada y_train: 0


In [10]:
# Ganti None/NaN dengan string kosong pada X_train dan X_test
X_train = [x if pd.notna(x) else "" for x in X_train]
X_test = [x if pd.notna(x) else "" for x in X_test]

kita perlu mengubah teks (patterns) menjadi representasi numerik. Kita akan menggunakan CountVectorizer dari sklearn untuk mengubah teks menjadi vektor.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Inisialisasi CountVectorizer
vectorizer = CountVectorizer()

# Mengubah data pelatihan menjadi vektor
X_train_vectorized = vectorizer.fit_transform(X_train)

# Mengubah data pengujian menjadi vektor
X_test_vectorized = vectorizer.transform(X_test)

# Menampilkan bentuk dari vektor
print(f'Bentuk data pelatihan: {X_train_vectorized.shape}')
print(f'Bentuk data pengujian: {X_test_vectorized.shape}')

Bentuk data pelatihan: (534, 222)
Bentuk data pengujian: (134, 222)


In [12]:
from sklearn.naive_bayes import MultinomialNB

# Inisialisasi model Naive Bayes
model = MultinomialNB()

# Melatih model
model.fit(X_train_vectorized, y_train)

# Menampilkan pesan bahwa model telah dilatih
print("Model telah dilatih.")

Model telah dilatih.


In [13]:
from sklearn.metrics import accuracy_score

# Memprediksi data pengujian
y_pred = model.predict(X_test_vectorized)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model: {accuracy * 100:.2f}%')

Akurasi model: 8.21%


In [17]:
import pandas as pd
import random

# Fungsi untuk mendapatkan respon berdasarkan pola
def get_response(input_text):
    # Filter baris berdasarkan pola yang cocok
    responses = data[data['Pattern'].str.contains(input_text, case=False, na=False)]['Response'].values
    if len(responses) > 0:
        # Pilih respon secara acak dari kolom response
        return random.choice(responses[0].split('|'))
    else:
        # Respon default jika tidak ada pola yang cocok
        return "Maaf, saya tidak mengerti apa yang Anda maksud."

# Fungsi chatbot untuk preprocessing input dan menghasilkan respon
def chatbot_response(input_text):
    # Preprocessing input pengguna
    input_text = input_text.lower()  # Ubah menjadi huruf kecil
    # Mendapatkan respon berdasarkan input
    response = get_response(input_text)
    return response

# Loop interaktif untuk menjalankan chatbot
print("Chatbot siap! Ketik 'quit' untuk keluar.")
while True:
    user_input = input("Anda: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Sampai jumpa!")
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")

Chatbot siap! Ketik 'quit' untuk keluar.
Anda: hallo
Chatbot: Halo, ada yang bisa saya bantu?
Anda: apa itu usus buntu?
Chatbot: Sakit yang mendadak muncul pada perut kanan bawah.
- Rasa sakit semakin parah saat batuk, berjalan, atau bergerak.
- Konstipasi atau diare.
- Mual dan muntah-muntah.
- Tidak nafsu makan.
- Perut kembung.
- Demam ringan yang semakin parah seiring memburuknya penyakit.
- Sering kentut.
- Pembengkakan dan nyeri tekan perut.
- Merasa lelah dan tidak punya energi.
- Demam tinggi
Anda: gejala usus buntu kronis
Chatbot: Usus buntu kronis dapat dikenali melalui ciri-ciri berikut:
- Sakit yang mendadak muncul pada perut kanan bawah
- Konstipasi atau diare
- Mual dan muntah-muntah
- Pembengkakan dan nyeri tekan perut
- Merasa lelah dan tidak punya energi
- Demam tinggi

Solusi untuk kondisi ini adalah dengan segera berkonsultasi dengan dokter. Dokter akan melakukan pemeriksaan fisik yang mendetail, ditambah tes penunjang seperti:
- Pemeriksaan darah dan urine
- USG
- R